## Data features


In [13]:
import os
import sys
import numpy as np
import typing
from matplotlib import pyplot as plt

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

import downloader.typing.bugswarm as typ_bugswarm
from preprocess.cmd.export_json import load_artifact

from consts import (
    ARTIFACTS_JSON_PATH,
    LOG_FILTERED_DIR,
    EXPORT_DIR,
    PASSED_LOG_NAME,
    FAILED_LOG_NAME,
    DIFF_NO_CTX_NAME,
)

art_path = os.path.join(os.getcwd(), "../..", ARTIFACTS_JSON_PATH)
data_src = load_artifact(art_path)
print(f"Loaded {len(data_src)} tasks")
data_hunks = np.load(
    os.path.abspath(os.path.join(os.getcwd(), "../..", EXPORT_DIR, "data.json"))
)


class Item(typing.TypedDict):
    len_passed: int
    len_failed: int
    len_filtered_passed: int
    len_filtered_failed: int
    lang: str
    num_hunk: int
    len_hunks_avg: float


res = {}  # type: dict[str, dict[str, int]]

Loaded 4478 tasks


FileNotFoundError: [Errno 2] No such file or directory: '/Users/mondo/Workspace/UW-Madison/CS561/project/mondo/bugswarm-data/data/exported/data.json'

In [ ]:
# length
# length-after filter
# length-delta
# per type/all
# hunk avg length
# hunk nums